In [1]:
import os
import glob
import pandas as pd
import plotly as py
import cufflinks as cf
import re
%matplotlib inline

In [2]:
base_path_to_csv = "/home/till/SharedFolder/trained_models/night-drive/weather_classifier/"
list_runs = ["without_cloudy", "with_cloudy"]
list_trials = ["train_A_over", "train_B_over", "train_C_over", "train_A_over_ganaug_025", "train_A_over_ganaug_050", "train_B_over_ganaug_025"]
list_selepochs = [[10, 10, 13, 10, 13, 10], # for each trial of run "without_cloudy"
                  [10, 10, 10, 10, 10, 10]] # for each trial of run "with_cloudy"
which_mode = "common"  # Options ["manual", "last", "best_train_dev", "best_f1"]
epoch_common = 40

In [3]:
df_sel_manual = pd.DataFrame()
df_all = pd.DataFrame()
for r, run in enumerate(list_runs):
    for t, trial in enumerate(list_trials):  # for each data set
        path_to_csv = os.path.join(base_path_to_csv,run,trial)
        csv_files = glob.glob(path_to_csv + "/*.csv")
        ### Combine CSVs into one DataFrame ###
        df = pd.DataFrame()
        for csv_file in csv_files:
            df = pd.concat([df, pd.read_csv(csv_file)], axis = 0)
            df = df.reset_index(drop = True)
        ### Some preprocessing ###
        #
        df = df.drop(columns = ["weights"])
        #
        df.data = df.data.apply(lambda x: x.split(os.sep)[-1].split(".json")[0].split("bdd100k_sorted_")[-1])
        df = df.sort_values(by = ["data", "epoch"])
        df = df.reset_index(drop = True)
        ### Load training log file ###
        train_log_file = glob.glob(path_to_csv + "/*.txt")
        assert(len(train_log_file) == 1)
        df_loss = pd.read_csv(train_log_file[0], sep = " ", header = None, skiprows = [0, 1])
        df_loss = df_loss.iloc[:, [1, 3, 7, 11]].rename(columns = {1: "epoch", 3: "batch", 7: "train_loss", 11: "train_dev_loss"})
        df_loss.epoch = df_loss.epoch.apply(lambda x: int(x.split("/")[0]))
        df_loss.batch = df_loss.batch.apply(lambda x: int(x.split("/")[0]))
        df_loss["global_batch"] = (df_loss.epoch - 1) * df_loss.batch.max() + df_loss.batch
        df_loss = df_loss.sort_values(by = ["epoch", "batch"]).set_index("global_batch", drop = True)
        ### Combine loss, scores
        df_combined = df_loss.groupby(by = ["epoch"]).median()
        df_combined = df_combined.drop(columns = ["batch"])
        
        # stash results for best manually selected epoch
        if which_mode == "manual":
            sel_epoch = list_selepochs[r][t]
        elif which_mode == "common":
            sel_epoch = epoch_common
            
        rw = df.query("epoch==@sel_epoch")
        rw.loc[:, "trial"] = list_trials[t]
        rw.loc[:, "run"] = list_runs[r]
        df_sel_manual = df_sel_manual.append(rw, ignore_index=True)
        # prmacro differences for each epoch
        # or all metrics using multi index
        df_all = pd.concat([df_all, df], axis=1)
        # stash results for best epoch according to: min train loss in train_dev
        
        # stash results for best epoch according to: f1_score
        which_score = "F1_score"
        

/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To ret

In [4]:
df_sel_manual.head(5)

,accuracy,data,epoch,f1_score_macro,f1_score_micro,f1_score_weighted,mcc,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro,run,trial
0,0.893041,test,40,0.791867,0.893041,0.892044,NaN,0.849390,0.945574,0.929549,0.968840,without_cloudy,train_A_over
1,0.930233,test_daytime,40,0.898486,0.930233,0.930539,NaN,0.956918,0.980116,0.986176,0.988961,without_cloudy,train_A_over
2,0.869474,test_night,40,0.663411,0.869474,0.866014,NaN,0.686719,0.913453,0.845422,0.951900,without_cloudy,train_A_over
3,0.908545,train_dev_A_over,40,0.908601,0.908545,0.908599,NaN,0.967581,0.966268,0.982222,0.980927,without_cloudy,train_A_over
4,0.889820,valid,40,0.783580,0.889820,0.889450,NaN,0.852354,0.950236,0.936420,0.971064,without_cloudy,train_A_over


In [5]:
### Create tables of summary stats of best epochs for every subplit
# rename train devs
df_sel_manual.data = df_sel_manual.data.apply(lambda x: re.sub(r"_[A-C]_[a-zA-Z0-9_]*", r"", x))

### Print results for best manual epochs gouped by split

In [6]:
# Selection (and order) of columns to be printed
cols2print = ["epoch", "trial", "run", 'accuracy','f1_score_macro', 'f1_score_micro', 'f1_score_weighted', 'pr_macro', 'pr_micro', 'roc_auc_macro', 'roc_auc_micro']

#### Valid-All

In [7]:
split = "valid"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "=== Results for best manual epoch ===")
df_cursplit.loc[:, cols2print].head(12)


=== valid === Results for best manual epoch ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,40,train_A_over,without_cloudy,0.889820,0.783580,0.889820,0.889450,0.852354,0.950236,0.936420,0.971064
1,40,train_B_over,without_cloudy,0.886598,0.792895,0.886598,0.889747,0.881026,0.953139,0.954105,0.973337
2,40,train_C_over,without_cloudy,0.885309,0.790260,0.885309,0.888790,0.864707,0.946349,0.950298,0.969590
3,40,train_A_over_ganaug_025,without_cloudy,0.882732,0.779181,0.882732,0.884311,0.856611,0.947808,0.941107,0.970216
4,40,train_A_over_ganaug_050,without_cloudy,0.877577,0.778588,0.877577,0.881690,0.853329,0.940512,0.941699,0.965437
5,40,train_B_over_ganaug_025,without_cloudy,0.909149,0.822871,0.909149,0.908971,0.882712,0.961221,0.953382,0.977627
6,40,train_A_over,with_cloudy,0.822500,0.738252,0.822500,0.822823,0.804790,0.882307,0.926545,0.952088
7,40,train_B_over,with_cloudy,0.829500,0.765341,0.829500,0.830894,0.836391,0.893457,0.938327,0.956270
8,40,train_C_over,with_cloudy,0.825500,0.758048,0.825500,0.827340,0.830144,0.892775,0.938936,0.956315
9,40,train_A_over_ganaug_025,with_cloudy,0.796500,0.718847,0.796500,0.799663,0.798448,0.854268,0.917707,0.941948


#### Valid-Night

In [8]:
split = "valid_night"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.head(12)


=== valid_night ===


,accuracy,data,epoch,f1_score_macro,f1_score_micro,f1_score_weighted,mcc,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro,run,trial
0,0.872632,valid_night,40,0.677948,0.872632,0.871957,NaN,0.712128,0.924982,0.867870,0.956553,without_cloudy,train_A_over
1,0.869474,valid_night,40,0.711523,0.869474,0.875541,NaN,0.791588,0.931796,0.911591,0.961416,without_cloudy,train_B_over
2,0.865263,valid_night,40,0.717173,0.865263,0.873756,NaN,0.797764,0.924146,0.917653,0.957276,without_cloudy,train_C_over
3,0.867368,valid_night,40,0.681170,0.867368,0.868530,NaN,0.725998,0.923877,0.881512,0.957387,without_cloudy,train_A_over_ganaug_025
4,0.852632,valid_night,40,0.682168,0.852632,0.860794,NaN,0.736310,0.913558,0.886798,0.949857,without_cloudy,train_A_over_ganaug_050
5,0.901053,valid_night,40,0.746900,0.901053,0.898420,NaN,0.799628,0.948645,0.911384,0.970728,without_cloudy,train_B_over_ganaug_025
6,0.829000,valid_night,40,0.611683,0.829000,0.825891,0.488646,0.632412,0.866353,0.839313,0.942095,with_cloudy,train_A_over
7,0.860000,valid_night,40,0.700336,0.860000,0.860600,0.604154,0.738884,0.900863,0.885558,0.957341,with_cloudy,train_B_over
8,0.844000,valid_night,40,0.680184,0.844000,0.847903,0.570673,0.736690,0.897807,0.893156,0.956999,with_cloudy,train_C_over
9,0.778000,valid_night,40,0.560864,0.778000,0.792269,0.442274,0.643137,0.813003,0.834050,0.926162,with_cloudy,train_A_over_ganaug_025


In [9]:
print("\n===", split, "Difference to A ===")
df_cursplit_diff = df_cursplit.copy()
for col in df_cursplit.columns:
    if not isinstance(df_cursplit.loc[0,col], str):
        df_cursplit_diff.loc[0:5,col] = df_cursplit_diff.loc[0:5,col] - df_cursplit.loc[0,col]
        df_cursplit_diff.loc[6:11,col] = df_cursplit_diff.loc[6:11,col] - df_cursplit.loc[6,col]
df_cursplit_diff.loc[:,cols2print].head(12)


=== valid_night Difference to A ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,0,train_A_over,without_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,train_B_over,without_cloudy,-0.003158,0.033575,-0.003158,0.003583,0.079461,0.006814,0.043721,0.004862
2,0,train_C_over,without_cloudy,-0.007368,0.039225,-0.007368,0.001799,0.085637,-0.000835,0.049783,0.000723
3,0,train_A_over_ganaug_025,without_cloudy,-0.005263,0.003222,-0.005263,-0.003427,0.013870,-0.001105,0.013642,0.000833
4,0,train_A_over_ganaug_050,without_cloudy,-0.020000,0.004220,-0.020000,-0.011163,0.024183,-0.011424,0.018928,-0.006697
5,0,train_B_over_ganaug_025,without_cloudy,0.028421,0.068952,0.028421,0.026463,0.087501,0.023663,0.043514,0.014175
6,0,train_A_over,with_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,train_B_over,with_cloudy,0.031000,0.088652,0.031000,0.034709,0.106472,0.034510,0.046246,0.015246
8,0,train_C_over,with_cloudy,0.015000,0.068501,0.015000,0.022012,0.104279,0.031454,0.053843,0.014904
9,0,train_A_over_ganaug_025,with_cloudy,-0.051000,-0.050819,-0.051000,-0.033622,0.010725,-0.053350,-0.005263,-0.015933


#### Test-All

In [10]:
split = "test"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.loc[:,cols2print].head(12)


=== test ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,40,train_A_over,without_cloudy,0.893041,0.791867,0.893041,0.892044,0.849390,0.945574,0.929549,0.968840
1,40,train_B_over,without_cloudy,0.893686,0.811577,0.893686,0.896373,0.881034,0.948017,0.943386,0.971062
2,40,train_C_over,without_cloudy,0.888531,0.799317,0.888531,0.892048,0.876282,0.945521,0.945515,0.969768
3,40,train_A_over_ganaug_025,without_cloudy,0.874356,0.766362,0.874356,0.876185,0.845613,0.943292,0.934093,0.966973
4,40,train_A_over_ganaug_050,without_cloudy,0.872423,0.773689,0.872423,0.876975,0.848532,0.935187,0.932805,0.962689
5,40,train_B_over_ganaug_025,without_cloudy,0.907216,0.814890,0.907216,0.906498,0.873301,0.954556,0.944380,0.974002
6,40,train_A_over,with_cloudy,0.842500,0.769192,0.842500,0.842002,0.827548,0.896343,0.928833,0.957352
7,40,train_B_over,with_cloudy,0.845500,0.783595,0.845500,0.847093,0.844906,0.894831,0.941317,0.959036
8,40,train_C_over,with_cloudy,0.849500,0.784015,0.849500,0.850207,0.845498,0.901672,0.942846,0.960884
9,40,train_A_over_ganaug_025,with_cloudy,0.797000,0.730927,0.797000,0.801120,0.814363,0.865872,0.925215,0.945385


#### Test-Night

In [11]:
split = "test_night"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.loc[:,cols2print].head(12)


=== test_night ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,40,train_A_over,without_cloudy,0.869474,0.663411,0.869474,0.866014,0.686719,0.913453,0.845422,0.951900
1,40,train_B_over,without_cloudy,0.872632,0.726097,0.872632,0.878429,0.784759,0.918725,0.888770,0.956871
2,40,train_C_over,without_cloudy,0.863158,0.700694,0.863158,0.870917,0.785979,0.912585,0.891864,0.953393
3,40,train_A_over_ganaug_025,without_cloudy,0.843158,0.635367,0.843158,0.846696,0.696527,0.914633,0.866091,0.951022
4,40,train_A_over_ganaug_050,without_cloudy,0.843158,0.663023,0.843158,0.852438,0.715014,0.898848,0.866074,0.943350
5,40,train_B_over_ganaug_025,without_cloudy,0.901053,0.742339,0.901053,0.898670,0.779434,0.936863,0.890424,0.965084
6,40,train_A_over,with_cloudy,0.851000,0.649404,0.851000,0.844845,0.674098,0.882963,0.846767,0.949371
7,40,train_B_over,with_cloudy,0.854000,0.666722,0.854000,0.851642,0.739498,0.885350,0.889316,0.954951
8,40,train_C_over,with_cloudy,0.858000,0.659759,0.858000,0.853619,0.743259,0.894478,0.886673,0.957056
9,40,train_A_over_ganaug_025,with_cloudy,0.763000,0.566210,0.763000,0.783602,0.648565,0.814285,0.841309,0.924054


In [12]:
print("\n===", split, "Difference to A ===")
df_cursplit_diff = df_cursplit.copy()
for col in df_cursplit.columns:
    if not isinstance(df_cursplit.loc[0,col], str):
        df_cursplit_diff.loc[0:5,col] = df_cursplit_diff.loc[0:5,col] - df_cursplit.loc[0,col]
        df_cursplit_diff.loc[6:11,col] = df_cursplit_diff.loc[6:11,col] - df_cursplit.loc[6,col]
df_cursplit_diff.loc[:,cols2print].head(12)


=== test_night Difference to A ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,0,train_A_over,without_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,train_B_over,without_cloudy,0.003158,0.062686,0.003158,0.012415,0.098040,0.005273,0.043348,0.004971
2,0,train_C_over,without_cloudy,-0.006316,0.037284,-0.006316,0.004903,0.099260,-0.000867,0.046442,0.001492
3,0,train_A_over_ganaug_025,without_cloudy,-0.026316,-0.028044,-0.026316,-0.019318,0.009808,0.001181,0.020670,-0.000879
4,0,train_A_over_ganaug_050,without_cloudy,-0.026316,-0.000387,-0.026316,-0.013576,0.028295,-0.014605,0.020653,-0.008551
5,0,train_B_over_ganaug_025,without_cloudy,0.031579,0.078928,0.031579,0.032656,0.092715,0.023410,0.045002,0.013183
6,0,train_A_over,with_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,train_B_over,with_cloudy,0.003000,0.017318,0.003000,0.006797,0.065400,0.002387,0.042548,0.005580
8,0,train_C_over,with_cloudy,0.007000,0.010355,0.007000,0.008774,0.069161,0.011516,0.039906,0.007685
9,0,train_A_over_ganaug_025,with_cloudy,-0.088000,-0.083194,-0.088000,-0.061243,-0.025533,-0.068678,-0.005458,-0.025317


In [13]:
# plot difference of pr_macro vs epoch
